In [11]:
import numpy as np
import networkx as nx
import string

np_map = np.loadtxt("simple-map-dungeon.txt").astype(int)
G=nx.Graph()

In [34]:
np_map

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0,

## Creation of the Graph

In [26]:
for y in range(np_map.shape[0]):
    for x in range(np_map.shape[1]):
        if np_map[y,x]:
            continue
        
        if y+1 < np_map.shape[0] and np_map[y+1,x] == 0:
            G.add_edge((x,y),(x,y+1))
        
        if y-1 >= 0 and  np_map[y-1,x] == 0:
            G.add_edge((x,y),(x,y-1))
        
        if x+1 < np_map.shape[1] and  np_map[y,x+1] == 0:
            G.add_edge((x,y),(x+1,y))
        
        if x-1 >= 0 and  np_map[y,x-1] == 0:
            G.add_edge((x,y),(x-1,y))
        

In [31]:
paths = nx.shortest_path(G)

In [36]:
def get_dist(G, v1, v2):
    return len(paths[v1][v2])

## 10.2 - K-medoids clustering

In [140]:
import random


centroids = []

for i in range(4):
    centroids.append(random.choice(list(G.nodes())))

print(centroids)
    
for epoch in range(50):
    clusters = [set() for _ in range(len(centroids))]
    # Loop over the nodes
    for node in G.nodes():
        # Get closest distance to centroid i
        
        min_dist = np.inf
        min_centr = ()
        
        for centr in centroids:
            dist = get_dist(G, node, centr)
            # Save min centroid
            if dist < min_dist:
                min_dist = dist
                min_centr = centr
        
        # Add node to correct cluster
        idx = centroids.index(min_centr)
        clusters[idx].add(node)

    for idx, clust_set in enumerate(clusters):
        min_dist = np.inf
        min_centr = ()
        for node in clust_set:
            sum_dist = sum(get_dist(G, node, node2) for node2 in clust_set)
            if sum_dist < min_dist:
                min_dist = sum_dist
                min_centr = node

        centroids[idx] = min_centr    
print(centroids)        

[(3, 5), (20, 5), (17, 12), (11, 20)]
[(4, 3), (17, 5), (14, 15), (5, 15)]


In [141]:
map_clustered = np_map.copy()
for y in range(np_map.shape[0]):
    for x in range(np_map.shape[1]):
        node = np_map[y,x]
        if node == 0:
            for idx, clust_set in enumerate(clusters):
                if (x,y) in clust_set:
                    map_clustered[y, x] = idx + 1
        else:
            map_clustered[y, x] = 0
map_clustered 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 0],
       [0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0],
       [0, 4, 4, 4, 4, 4,

## 10.3 - Spectral clustering